In [148]:
import bs4
import requests
import pandas as pd
import os 
import re
from itertools import product
from time import sleep

In [149]:
def article_clean(articles):
    article = articles.text
    article = article.replace('\n','').strip()
    article = article.replace('\t','').strip()
    article = article.replace('\xa0','').strip()
    article = article.replace('\r','').strip() 
    article = re.sub(r'\(NASDAQ: [A-Z]+\)', '', article).strip()
    article = re.sub(r'\(NYSE: [A-Z]+\)', '', article).strip()
    article = article.replace('Continue reading','').strip()
    article = article.replace('  ','').strip()
    return article

In [150]:
def get_article(url) -> str:
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    articles = soup.find('div', {'class': 'col-xs-12 news-content no-padding'})
    if articles is None:
        return ''
    if articles.find("span",{"class":"btn-more"}) is None:
        return article_clean(articles)
    articles.find("span",{"class":"btn-more"}).decompose()
    return article_clean(articles)

In [151]:
get_article("https://markets.businessinsider.com/news/stocks/better-buy--starbucks-vs--costco-12080493")

"Starbucks and Costco Wholesale both withstood the S&P 500's 14% decline over the past 12 months. Starbucks' stock rose 7%, while Costco experienced a mild decline of 4%. Let's see why these two S&P 500 companies outperformed so many of their peers -- and if one is a more promising investment right now.At first glance, Starbucks might seem like a risky investment. The coffee chain is heavily exposed to inflation, which drives up its costs and curbs the market's appetite for its pricey beverages. And the COVID-19 lockdowns in China affected its second-largest market after the United States.Starbucks also faces unionization demands in the U.S., while founder Howard Schultz -- who returned as its interim CEO last year -- is in the process of handing the reins to its new CEO, Laxman Narasimhan."

In [152]:
def get_articles(company:str,page_lenght:int,sleeptime:int = 1,page_number:int = 1) -> tuple:
    result = []
    try:
        for page in range(page_number,page_lenght+1):
            url = f"https://markets.businessinsider.com/news/{company}?p={page}&"
            response = requests.get(url)
            news_header = "https://markets.businessinsider.com"
            soup = bs4.BeautifulSoup(response.text, 'html.parser')
            if "No news found" in soup.find('main', {'class': 'site-content'}).text:
                break
            articles = soup.find('main', {'class': 'site-content'}).find_all('div', {'class': 'latest-news__story'})
            for article in articles:
                page = round(len(result) // 50) 
                print(f"Page: {page} - Articles: {len(result)}", end="\r")
                source = article.find('span', {'class': 'latest-news__source'}).text
                if source.lower() != "seeking alpha":
                    link = news_header+article.find('a').get('href')
                    title = article.find('a').text
                    date = article.find('time', {'class': 'latest-news__date'}).text
                    article = get_article(link)
                    result.append([source, link, title, article,date])	
                sleep(sleeptime)
    except Exception as e:
        if e == ConnectionError:
            sleepti = sleeptime + 5
            page_num = round(len(result) // 50)
            get_articles(company,page_lenght,sleepti,page_num)
        print(e)
    return result

In [154]:
damn = get_articles("cost",1)

In [145]:
len(damn)

45

In [146]:
df = pd.DataFrame(damn, columns=['Source','Link', 'Title', 'Article','Date'])
df.to_csv('costco.csv', index=False)